# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
#used mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray

In [7]:
# confirm that our new database was created
print("Databases:", mongo.list_database_names())

Databases: ['admin', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
print(db.list_collection_names())

['establishments']


In [10]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [11]:
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67c60578bac911208fd63e4d'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None

In [12]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "AddressLine4": "Greenwich",
    "AddressLine2": "",
    "PostCode": "",
    "RatingValue": None, 
    "LocalAuthorityName": "Greenwich",
}

In [14]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_restaurant)


InsertOneResult(ObjectId('67c60a8ab8bf1f1cc867d612'), acknowledged=True)

In [15]:
# Check that the new restaurant was inserted
inserted_restaurant = db.establishments.find_one({"BusinessName": "Penang Flavours"})
print(inserted_restaurant)

{'_id': ObjectId('67c6082e134cfcae3d4c1cd3'), 'BusinessName': 'Penang Flavours', 'AddressLine4': 'Greenwich', 'AddressLine2': '', 'PostCode': '', 'RatingValue': None, 'LocalAuthorityName': 'Greenwich', 'BusinessTypeID': 1, 'geocode': {'longitude': None, 'latitude': None}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [16]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

print(business_type)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [17]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)


UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [18]:
# Confirm that the new restaurant was updated
updated_restaurant = db.establishments.find_one({"BusinessName": "Penang Flavours"})
print(updated_restaurant)

{'_id': ObjectId('67c6082e134cfcae3d4c1cd3'), 'BusinessName': 'Penang Flavours', 'AddressLine4': 'Greenwich', 'AddressLine2': '', 'PostCode': '', 'RatingValue': None, 'LocalAuthorityName': 'Greenwich', 'BusinessTypeID': 1, 'geocode': {'longitude': None, 'latitude': None}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [19]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of establishments in Dover: {dover_count}")

Number of establishments in Dover: 994


In [20]:
# Delete all documents where LocalAuthorityName is "Dover"
db.establishments.delete_many({"LocalAuthorityName": "Dover"})


DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [21]:
# Check if any remaining documents include Dover
remaining_dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining establishments in Dover: {remaining_dover_count}")

Remaining establishments in Dover: 0


In [22]:
# Check that other documents remain with 'find_one'
remaining_dover_document = db.establishments.find_one({"LocalAuthorityName": "Dover"})
print("Example of remaining Dover document:", remaining_dover_document)

Example of remaining Dover document: None


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [23]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {},
    [
        {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
        {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
    ]
)


UpdateResult({'n': 77573, 'nModified': 38787, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [26]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [27]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many(
    {"RatingValue": {"$type": "string", "$ne": None}}, 
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


UpdateResult({'n': 69388, 'nModified': 69388, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [28]:
# Check that the coordinates and rating value are now numbers
sample_doc = db.establishments.find_one({}, {"geocode.longitude": 1, "geocode.latitude": 1, "RatingValue": 1, "_id": 0})

pprint(sample_doc)

print("Longitude Type:", type(sample_doc.get("geocode", {}).get("longitude")))
print("Latitude Type:", type(sample_doc.get("geocode", {}).get("latitude")))
print("RatingValue Type:", type(sample_doc.get("RatingValue")))


{'RatingValue': 5, 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
Longitude Type: <class 'float'>
Latitude Type: <class 'float'>
RatingValue Type: <class 'int'>
